In [ ]:
import sys
import os

# 获取当前 notebook 所在的绝对路径
current_path = os.getcwd()
if current_path not in sys.path:
    sys.path.append(current_path)

In [1]:
from pyspark_common import ClickHouseSparkManager
from setup import DBSetupManager
from bronze import Bronze
from silver import Silver
from gold import Gold

✅ Preloaded Suite: straitstimes_news_bz_suite


In [2]:
once = True 
processing_time = "5 seconds"

In [ ]:
manager = ClickHouseSparkManager(
    db_user="default",
    db_password="123456",
    db_host="clickhouse-server",
    db_port="8123",
    db_name="news" # 建议与 Docker 镜像参数对齐，使用 news
)

# 2. 启动 Spark
spark = manager.create_session("NewsAnalysisProject")

26/01/06 16:26:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# 3. 初始化 DBsetup (传入与上面一致的配置)
db_conf = {
    "db_user": "default",
    "db_password": "123456",
    "db_host": "clickhouse-server",
    "db_name": "news"
}

DBsetup = DBSetupManager(spark_session=spark, **db_conf)

In [5]:
bronze = Bronze(spark, DBsetup)

📂 数据文件夹: /Users/luhui/Documents/github/08_straitstimes_news_ELT/data


In [6]:
bronze.consume(once, processing_time)


Starting bronze layer consumption ...


26/01/06 16:26:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


🔒 Batch 0: Processing straitstimes_news_bz...
🚀 Batch 0: Running GX for straitstimes_news_bz...


26/01/06 16:27:00 WARN CacheManager: Asked to cache already cached data.


Calculating Metrics:   0%|          | 0/34 [00:00<?, ?it/s]

🚀 正在通过 HTTP 协议分布式写入到 data_quality_quarantine (列: ['table_name', 'gx_batch_id', 'violated_rules', 'raw_data'])...
✅ 写入完成
🚀 正在通过 HTTP 协议分布式写入到 straitstimes_news_bz (列: ['title', 'publish_date', 'update_date', 'img_url', 'caption_text', 'tags_list', 'full_article', 'url', 'load_time'])...
✅ 写入完成
⚠️ Batch 0: straitstimes_news_bz FAILED. Quarantined 1 rows.
Completed bronze layer consumtion 4 seconds


In [7]:
silver = Silver(spark, DBsetup)

In [8]:
silver.upsert()


2026-01-06 16:27:03 🚀 开始执行 Silver 层增量同步...
🔍 当前 Silver 表水位线: 1970-01-01 00:00:00
Checking for duplicates OK
Checking for all-null rows OK
Checking for nulls on string columns OK
Checking for nulls on numeric columns OK
🚀 正在通过 HTTP 协议分布式写入到 news.straitstimes_news_sl (列: ['title', 'publish_date', 'update_date', 'img_url', 'caption_text', 'tags_list', 'full_article', 'url', 'load_time', 'update_time'])...
✅ 写入完成
✅ 成功同步 69 条数据至 Silver 层。
✨ 本次批处理耗时: 1 秒


In [9]:
gold = Gold(spark, DBsetup)

In [10]:
gold.upsert()


🚀 开始加工 Gold 层数据...
🚀 正在通过 HTTP 协议分布式写入到 news.fact_news_gl (列: ['url', 'title', 'publish_date', 'update_date', 'img_url', 'caption_text', 'full_article', 'publish_timekey', 'update_timekey', 'update_time'])...


✅ 写入完成
✅ Fact 表成功写入 69 条
🚀 正在通过 HTTP 协议分布式写入到 news.dim_tags_gl (列: ['url', 'tag', 'update_time'])...


✅ 写入完成
✅ Dim Tags 表成功写入 231 条


In [11]:
spark.stop()